In [3]:
import os
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.io import mmread, mmwrite
import torch
from scipy.sparse import csr_matrix, csc_matrix
from gflownet.utils import resize_sparse_tensor, log_memory_usage, malloc_usage, calculate_reward, update_edges_and_convert_to_sparse, trajectory_balance_loss, decompose_ilu_and_create_linear_operator
from typing import List
from gflownet.dataset import *
import gc
from tqdm import tqdm
from gflownet.validate import solve_with_gmres, load_mtx_file, load_vector_mtx, load_mtx_file_csc
from gflownet.utils import torch_sparse_to_csr,  decompose_ilu_and_create_linear_operator, apply_ilu_preconditioner, custom_solve_with_modified_LU
from scipy.sparse import csr_matrix, triu, tril
from scipy.sparse.linalg import gmres, LinearOperator, spilu, splu

In [2]:
path = '/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/hangGlider_3/'
vector_path = '/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/hangGlider_3/'
root_filenames = ['hangGlider_3']

for i in root_filenames:
    filename = path + i + '.mtx'
    vector_file = vector_path + i + '_b.mtx'
    A = load_mtx_file_csc(filename)
    b = load_vector_mtx(vector_file)
    ilu = spilu(A)
    L = ilu.L.copy()
    U = ilu.U.copy()
    perm_c = ilu.perm_c
    perm_r = ilu.perm_r
    M_x_orig = lambda x: ilu.solve(x)
    M_orig = spla.LinearOperator(A.shape, M_x_orig)
    no_p_x, no_p_res, no_p_iters, no_p_time = solve_with_gmres(A, b)
    pre_x, pre_residuals, pre_num_iterations, pre_elapsed_time = solve_with_gmres(A, b, M=M_orig)
    M_custom = LinearOperator(A.shape, lambda x: custom_solve_with_modified_LU(x, L, U, perm_r=perm_r, perm_c=perm_c))
    Mcustom_x, Mcustom_res, Mcustom_iters, Mcustom_time = solve_with_gmres(A, b, M=M_custom)
    print("No Preconditioner")
    print(no_p_iters)
    print(no_p_time)
    print("Default Solver")
    print(pre_num_iterations)
    print(pre_elapsed_time)
    print("Custom Solver")
    print(Mcustom_iters)
    print(Mcustom_time)

    


GMRES did not converge. Exit code: 20000
GMRES converged successfully.
Forward Substitution time
0.13790321350097656


/Users/tonylizza/opt/anaconda3/envs/ML2/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:590: SparseEfficiencyWarning: CSR matrix format is required. Converting to CSR matrix.
  warn('CSR matrix format is required. Converting to CSR matrix.',


Backward Solve Time
0.13249516487121582
Pc Time
0.0006740093231201172
Z Time
4.291534423828125e-05
Forward Substitution time
0.136246919631958
Backward Solve Time
0.11788511276245117
Pc Time
0.0006902217864990234
Z Time
4.220008850097656e-05
Forward Substitution time
0.12784481048583984
Backward Solve Time
0.1152029037475586
Pc Time
0.0005352497100830078
Z Time
4.076957702636719e-05
Forward Substitution time
0.11993288993835449
Backward Solve Time
0.12003087997436523
Pc Time
0.0005669593811035156
Z Time
4.506111145019531e-05
Forward Substitution time
0.1259479522705078
Backward Solve Time
0.11886286735534668
Pc Time
0.0007658004760742188
Z Time
4.506111145019531e-05
Forward Substitution time
0.12872600555419922
Backward Solve Time
0.1193852424621582
Pc Time
0.0005910396575927734
Z Time
4.291534423828125e-05
Forward Substitution time
0.13630080223083496
Backward Solve Time
0.12265706062316895
Pc Time
0.000537872314453125
Z Time
3.600120544433594e-05
Forward Substitution time
0.126885890

In [4]:
import torch_geometric

In [8]:
import os
import scipy.io
import scipy.sparse

def get_max_nnz_from_mtx_files(directory):
    """
    Function to get the maximum nnz (number of non-zero entries) for .mtx files in a directory.
    
    Args:
        directory (str): The directory containing .mtx files.
    
    Returns:
        int: The maximum nnz across all .mtx files in the directory.
    """
    max_nnz = 0
    
    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.mtx'):
            file_path = os.path.join(directory, filename)
            
            # Load the matrix from the .mtx file
            matrix = scipy.io.mmread(file_path)
            print(matrix.nnz)
            # Check if the matrix is sparse and get the number of non-zero elements
            if scipy.sparse.issparse(matrix):
                nnz = matrix.nnz  # Number of non-zero elements in the matrix
            else:
                nnz = (matrix != 0).sum()  # If dense, count non-zero elements
            
            # Update the maximum nnz if current matrix has more non-zero elements
            if nnz > max_nnz:
                max_nnz = nnz
                
    return max_nnz




In [9]:
# Example usage
directory = 'data/medium_ILU/ilu_matrices'
max_nnz = get_max_nnz_from_mtx_files(directory)
print(f"The maximum nnz in the .mtx files is: {max_nnz}")

4652
2315
62470
38515
4657
5565
4530
4652
12322
12322
5011
The maximum nnz in the .mtx files is: 62470


In [10]:
# Example usage
directory = 'data/medium_ILU/matrices'
max_nnz = get_max_nnz_from_mtx_files(directory)
print(f"The maximum nnz in the .mtx files is: {max_nnz}")

2480
28726
2500
7422
4285
4285
4285
2500
4285
2480
3474
The maximum nnz in the .mtx files is: 28726
